In [0]:
#Scraping der Daten aus der Open Source Library der National Gallery of Denmark (SMK)
import requests
import torch
from PIL import Image
import io 
from IPython.display import clear_output
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
respone = requests.get('https://api.smk.dk/api/v1/art/all_ids')

In [0]:
import json
ids = json.loads(respone.content)['objectIDs']

In [5]:
print(len(ids))

79002


In [0]:
import os
path = "/content/drive/My Drive/KI-Schule/Projekt/Daten/SMK/"
if not os.path.exists(path):
    os.makedirs(path)

In [0]:
rights_file = '/content/drive/My Drive/KI-Schule/Projekt/Daten/is_public.json'
processed_ids_file = '/content/drive/My Drive/KI-Schule/Projekt/Daten/processed_ids.json'


In [0]:
links = {}
if os.path.exists(processed_ids_file):
    is_public = json.load(open(rights_file))
    processed_ids = json.load(open(processed_ids_file))
else:
    processed_ids = []
    is_public = {}

download_image = False
chunk = set(ids).difference(processed_ids)

to_process = len(chunk)
links_found = 0
for ix, oid in enumerate(chunk):  
    manifest = requests.get(f'https://api.smk.dk/api/v1/art/?object_number={oid}')
    manifest = json.loads(manifest.content)
    for m in manifest['items']:
        l = m.get('image_thumbnail', False)
        if l:
            try:
                onr = m['object_number']
                image_name = f"{onr.replace('/','_')}.jpg"
                links_found += 1
                if download_image:
                    image = requests.get(l)
                    image = Image.open(io.BytesIO(image.content))
                    image.save(os.path.join(path, image_name))
                is_public[image_name] = m.get('public_domain', None)
            except:
                print('failed to read image')
    processed_ids.append(oid)
    if ix%100 == 0:
        json.dump(is_public, open(rights_file, 'w'))
        json.dump(processed_ids, open(processed_ids_file, 'w'))
        clear_output(wait=True)
        print(f'{to_process} ids need processing, {links_found} links extracted')
    to_process -= 1